## Importing all of the needed libraries

In [1]:
import pandas as pd
import numpy as np
import os
import bs4
from bs4 import BeautifulSoup
# import httpx
from pyPodcastParser.Podcast import Podcast
import requests as r
import gzip
from io import BytesIO
from pandas.core.common import flatten
from collections import OrderedDict

## Getting all of the podcast ID's and the podcast shows

In [2]:
apple_podcast_url = "http://podcasts.apple.com/sitemaps_podcasts_index_1.xml"
apple_podcast_url_get = r.get(apple_podcast_url)
print(apple_podcast_url_get)
soup = BeautifulSoup(apple_podcast_url_get.text, features="xml")

<Response [200]>


In [3]:
%%time
list_of_sitemaps = [str(i).replace('<loc>','').replace('</loc>','') for i in soup.find_all('loc')]
podcast_shows_apple_id = []

def get_show_id(i):
    return str(i).replace('<loc>','').replace('</loc>','').rsplit('/', 1)[-1].replace('id','')

for i, sitemap in enumerate(list_of_sitemaps):
    with gzip.open(BytesIO(r.get(sitemap).content), 'r') as f:
        file_content = f.read()
        soup = BeautifulSoup(file_content, features="xml")
        podcast_shows_list_id = [get_show_id(i) for i in soup.find_all('loc')]
        podcast_shows_apple_id.append(podcast_shows_list_id)
               
podcast_shows_apple_id_final = list(set(list(flatten(podcast_shows_apple_id))))
df = pd.DataFrame({'podcast_id': podcast_shows_apple_id_final})
df.to_csv(os.path.join('..','data','podcast_id.csv'),index=False)

CPU times: user 6min 6s, sys: 3.09 s, total: 6min 9s
Wall time: 9min 8s


In [4]:
podcast_id_df = pd.read_csv(os.path.join('..','data','podcast_id.csv'))
print(podcast_id_df.shape)
podcast_id_df

(753559, 1)


,podcast_id
0,1623108919
1,1439038040
2,1652767154
3,1567538603
4,1650145244
...,...
753554,1585464065
753555,1638067536
753556,1588720898
753557,1469041262


Stick to the sample 10,000 podcast shows

In [5]:
%%time

def get_podcast_json_data(podcast_id):
    itunes_api = f"https://itunes.apple.com/lookup?id={str(podcast_id)}&media=podcast"
    itunes_podcast_json = r.get(itunes_api).json()
    return itunes_podcast_json['results'][0]


def get_podcast_data(df):
    list_of_podcast_data = []

    list_of_podcasts_extra = []

    for pod_id in df.podcast_id.to_list()[:10000]:
        try:
            itunes_api = f"https://itunes.apple.com/lookup?id={str(pod_id)}&media=podcast"
            itunes_podcast_json = r.get(itunes_api).json()
            list_of_podcast_data.append(itunes_podcast_json['results'][0])
        except:
            list_of_podcasts_extra.append([i, pod_id])
            pass

    df_1 = pd.DataFrame(list_of_podcast_data)
    return df_1

podcast_show_df = get_podcast_data(podcast_id_df)
podcast_show_df

CPU times: user 4min 24s, sys: 34.4 s, total: 4min 58s
Wall time: 36min 24s


,wrapperType,kind,collectionId,trackId,artistName,collectionName,trackName,collectionCensoredName,trackCensoredName,collectionViewUrl,...,primaryGenreName,contentAdvisoryRating,artworkUrl600,genreIds,genres,artistId,artistViewUrl,trackRentalPrice,trackHdPrice,trackHdRentalPrice
0,track,podcast,1623108919,1623108919,Anthony Harris & JJ Sweatt,Triple Double Watch,Triple Double Watch,Triple Double Watch,Triple Double Watch,https://podcasts.apple.com/us/podcast/triple-d...,...,Sports,Clean,https://is4-ssl.mzstatic.com/image/thumb/Podca...,"[1545, 26]","[Sports, Podcasts]",NaN,NaN,NaN,NaN,NaN
1,track,podcast,1439038040,1439038040,Panic State Studios,Movies & Brews,Movies & Brews,Movies & Brews,Movies & Brews,https://podcasts.apple.com/us/podcast/movies-b...,...,Film Reviews,Explicit,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"[1563, 26, 1309]","[Film Reviews, Podcasts, TV & Film]",NaN,NaN,NaN,NaN,NaN
2,track,podcast,1652767154,1652767154,Tyler Evertsen,D2D Sales: Tips & Tricks,D2D Sales: Tips & Tricks,D2D Sales: Tips & Tricks,D2D Sales: Tips & Tricks,https://podcasts.apple.com/us/podcast/d2d-sale...,...,Marketing,Clean,https://is1-ssl.mzstatic.com/image/thumb/Podca...,"[1492, 26, 1321]","[Marketing, Podcasts, Business]",NaN,NaN,NaN,NaN,NaN
3,track,podcast,1567538603,1567538603,deCatólicos,Liturgia de las Horas,Liturgia de las Horas,Liturgia de las Horas,Liturgia de las Horas,https://podcasts.apple.com/us/podcast/liturgia...,...,Religion,Clean,https://is5-ssl.mzstatic.com/image/thumb/Podca...,"[1532, 26, 1314]","[Religion, Podcasts, Religion & Spirituality]",NaN,NaN,NaN,NaN,NaN
4,track,podcast,1650145244,1650145244,Sandra Kriegelstein und Juliane Kriegelstein-P...,Seite 7 - Der Bücher-Podcast,Seite 7 - Der Bücher-Podcast,Seite 7 - Der Bücher-Podcast,Seite 7 - Der Bücher-Podcast,https://podcasts.apple.com/us/podcast/seite-7-...,...,Leisure,Clean,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"[1502, 26, 1305]","[Leisure, Podcasts, Kids & Family]",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9932,track,podcast,1051062557,1051062557,National Museum of the U.S. Air Force,Museum Lecture Series,Museum Lecture Series,Museum Lecture Series,Museum Lecture Series,https://podcasts.apple.com/us/podcast/museum-l...,...,Government,Clean,https://is1-ssl.mzstatic.com/image/thumb/Podca...,"[1511, 26]","[Government, Podcasts]",219753981.0,https://podcasts.apple.com/us/artist/dvids/219...,NaN,NaN,NaN
9933,track,podcast,1530637249,1530637249,Rádio Escola - EBM Mâncio Costa,Rádio Escola _ EBM Mâncio Costa,Rádio Escola _ EBM Mâncio Costa,Rádio Escola _ EBM Mâncio Costa,Rádio Escola _ EBM Mâncio Costa,https://podcasts.apple.com/us/podcast/r%C3%A1d...,...,Education,Clean,https://is3-ssl.mzstatic.com/image/thumb/Podca...,"[1304, 26]","[Education, Podcasts]",NaN,NaN,NaN,NaN,NaN
9934,track,podcast,1546206065,1546206065,Český rozhlas,Vize z krize,Vize z krize,Vize z krize,Vize z krize,https://podcasts.apple.com/us/podcast/vize-z-k...,...,Society & Culture,Clean,https://is3-ssl.mzstatic.com/image/thumb/Podca...,"[1324, 26]","[Society & Culture, Podcasts]",NaN,NaN,NaN,NaN,NaN
9935,track,podcast,1554832683,1554832683,lindsey benson,carcast lol,carcast lol,carcast lol,carcast lol,https://podcasts.apple.com/us/podcast/carcast-...,...,Personal Journals,Clean,https://is2-ssl.mzstatic.com/image/thumb/Podca...,"[1302, 26, 1324]","[Personal Journals, Podcasts, Society & Culture]",NaN,NaN,NaN,NaN,NaN


In [6]:
podcast_show_df.columns

Index(['wrapperType', 'kind', 'collectionId', 'trackId', 'artistName',
       'collectionName', 'trackName', 'collectionCensoredName',
       'trackCensoredName', 'collectionViewUrl', 'feedUrl', 'trackViewUrl',
       'artworkUrl30', 'artworkUrl60', 'artworkUrl100', 'collectionPrice',
       'trackPrice', 'collectionHdPrice', 'releaseDate',
       'collectionExplicitness', 'trackExplicitness', 'trackCount',
       'trackTimeMillis', 'country', 'currency', 'primaryGenreName',
       'contentAdvisoryRating', 'artworkUrl600', 'genreIds', 'genres',
       'artistId', 'artistViewUrl', 'trackRentalPrice', 'trackHdPrice',
       'trackHdRentalPrice'],
      dtype='object')

In [7]:
podcast_show_df.to_parquet(os.path.join('..','data','podcast_show_data.parquet'), index=False)

In [8]:
feed_url_list = podcast_show_df.dropna(subset=['feedUrl'])['feedUrl'].to_list()

In [9]:
def get_pod_ep_dict(rss):
    try:
        podcast = Podcast(r.get(rss).content)
        return [i.to_dict() for i in podcast.items]
    except:
        pass

In [10]:
# test = podcast_show_df[['feedUrl']].iloc[0:3]['feedUrl'].to_list()

In [11]:
# test

In [12]:
test_1 = list(map(get_pod_ep_dict, test))

NameError: name 'test' is not defined

In [23]:
def flatten_list_of_list_of_dict(lst):
    lst_test = []
    try:
        for i in lst:
            for j in i:
                lst_test.append(j)
    except:
        pass
    
    return lst_test

In [24]:
%%time
podcast_episode_list = flatten_list_of_list_of_dict(list(map(get_pod_ep_dict, feed_url_list)))

/Users/aymansulaiman/miniforge3/envs/podcherry/lib/python3.8/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


CPU times: user 19min 32s, sys: 27.6 s, total: 20min
Wall time: 1h 23min 41s


In [25]:
print(podcast_episode_list)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [26]:
podcast_episode_df = pd.DataFrame(podcast_episode_list)

In [27]:
podcast_episode_df

,author,comments,creative_commons,enclosure_url,enclosure_type,enclosure_length,guid,itunes_author_name,itunes_block,itunes_closed_captioned,itunes_duration,itunes_explicit,itune_image,itunes_order,itunes_subtitle,itunes_summary,link,published_date,title
0,None,None,None,https://media.rss.com/tripledoublewatch/202205...,audio/mpeg,21789862.0,77358eba-7801-4fb0-9674-095d712585ad,None,False,None,1191,no,https://media.rss.com/tripledoublewatch/202205...,None,None,Interview with Prince-Marshon Merriel.,None,"Mon, 09 May 2022 03:04:36 GMT",Prince-Marshon Interview
1,None,https://moviesandbrews.podbean.com/e/nope/#com...,None,https://mcdn.podbean.com/mf/web/8wmnft/M_B_NOP...,audio/mpeg,81838054.0,moviesandbrews.podbean.com/366317d0-a5f6-3574-...,Panic State Studios,False,None,56:49,yes,https://deow9bq0xqvbj.cloudfront.net/image-log...,None,The residents of a lonely gulch in inland Cali...,The residents of a lonely gulch in inland Cali...,None,"Fri, 05 Aug 2022 00:01:00 -0700",Nope.
2,None,https://moviesandbrews.podbean.com/e/the-gray-...,None,https://mcdn.podbean.com/mf/web/83xmjg/M_B_The...,audio/mpeg,78388257.0,moviesandbrews.podbean.com/8170d1ef-99a9-397f-...,Panic State Studios,False,None,54:25,yes,https://deow9bq0xqvbj.cloudfront.net/image-log...,None,"After a 3.5yr break, Ryan Gosling is back! He ...","After a 3.5yr break, Ryan Gosling is back! He ...",None,"Fri, 05 Aug 2022 00:01:00 -0700",The Gray Man
3,None,https://moviesandbrews.podbean.com/e/thor-love...,None,https://mcdn.podbean.com/mf/web/snw69u/M_B_Tho...,audio/mpeg,92411085.0,moviesandbrews.podbean.com/e194410c-6388-3cbe-...,Panic State Studios,False,None,01:04:09,yes,https://deow9bq0xqvbj.cloudfront.net/image-log...,None,Thor's back and once again going through a mid...,Thor's back and once again going through a mid...,None,"Thu, 21 Jul 2022 00:01:00 -0700",Thor: Love and Thunder
4,None,https://moviesandbrews.podbean.com/e/elvis/#co...,None,https://mcdn.podbean.com/mf/web/xmgapd/M_B_Elv...,audio/mpeg,107294623.0,moviesandbrews.podbean.com/acc49d56-d4d9-3a03-...,Panic State Studios,False,None,01:14:30,yes,https://deow9bq0xqvbj.cloudfront.net/image-log...,None,Austin Butler stars as the King himself in Baz...,Austin Butler stars as the King himself in Baz...,None,"Sun, 17 Jul 2022 00:00:00 -0700",Elvis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26076,None,https://ozarkshaintsnhooch.podbean.com/e/ozark...,None,https://mcdn.podbean.com/mf/web/zuuukf/OHHNHEp...,audio/mpeg,170513994.0,ozarkshaintsnhooch.podbean.com/9b21f29b-35b2-3...,Ozarks Haints N Hooch,False,None,01:11:03,yes,https://deow9bq0xqvbj.cloudfront.net/ep-logo/p...,None,Dina hosts Dawn’s big one about the Ozark Howl...,Dina hosts Dawn’s big one about the Ozark Howl...,None,"Tue, 01 Sep 2020 07:00:00 -0400",Ozarks Haints N Hooch Episode 5 - The Ozark Ho...
26077,None,https://ozarkshaintsnhooch.podbean.com/e/ozark...,None,https://mcdn.podbean.com/mf/web/2sgejv/OHHEp46...,audio/mpeg,114490716.0,ozarkshaintsnhooch.podbean.com/f444948e-c660-3...,Ozarks Haints N Hooch,False,None,47:42,yes,https://deow9bq0xqvbj.cloudfront.net/ep-logo/p...,None,Pam hosts Deidre’s big one on the Crescent Hot...,Pam hosts Deidre’s big one on the Crescent Hot...,None,"Sat, 15 Aug 2020 07:00:00 -0400",Ozarks Haints N Hooch - Episode 4 The Crescent...
26078,None,https://ozarkshaintsnhooch.podbean.com/e/ozark...,None,https://mcdn.podbean.com/mf/web/rhd1xz/OHNHEp3...,audio/mpeg,105218298.0,ozarkshaintsnhooch.podbean.com/5c1289cb-86a1-5...,Ozarks Haints N Hooch,False,None,43:50,yes,https://deow9bq0xqvbj.cloudfront.net/ep-logo/p...,None,"Deidre hosts Pam’s big one on the Blue Man, Da...","Deidre hosts Pam’s big one on the Blue Man, Da...",None,"Sat, 01 Aug 2020 07:00:00 -0400",Ozarks Haints N Hooch Episode 3 - Bigfoot in t...
26079,None,https://ozarkshaintsnhooch.podbean.com/e/ozark...,None,https://mcdn.podbean.com/mf/web/0j5o4b/OHHEp2e...,audio/mpeg,96718043.0,ozarkshaintsnhooch.podbean.com/07413f50-dd20-5

In [28]:
podcast_episode_df.to_parquet(os.path.join('..','data','podcast_episode_data.parquet'), index=False)